In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
train_df = pd.read_csv('/content/preprocesstrain.tsv',sep='\t')
test_df = pd.read_csv('/content/preprocesstest.tsv',sep='\t')

id_vars = ['guid/image']
label_vars = ['gt_label']
gt_vars = []
cocov1_vars = []
kitti_vars = []
incep_vars = []
for col in train_df.columns:
    if col.startswith('gt') and col != 'gt_label':
        gt_vars.append(col)
    elif col.startswith('cocov1'):
        cocov1_vars.append(col)
    elif col.startswith('kitti'):
        kitti_vars.append(col)
    elif col.startswith('incep'):
        incep_vars.append(col)
    
print (len(gt_vars)+len(cocov1_vars)+len(kitti_vars)+len(incep_vars)+len(id_vars)+len(label_vars) ==len(train_df.columns))

train_X = train_df[cocov1_vars + kitti_vars + incep_vars]
train_y = train_df[label_vars]
test_X = test_df[cocov1_vars + kitti_vars + incep_vars]

True


In [ ]:
# Splitting the data for model training#
shuffle_idx = np.arange(len(train_X))
np.random.seed(2018)
np.random.shuffle(shuffle_idx)
split = int(0.8*len(shuffle_idx))
dev_X = train_X.iloc[shuffle_idx[:split],:]
val_X = train_X.iloc[shuffle_idx[split:],:]
dev_y = train_y.iloc[shuffle_idx[:split],:]
val_y = train_y.iloc[shuffle_idx[split:],:]
print(dev_X.shape, val_X.shape, test_X.shape)

(6058, 30) (1515, 30) (2631, 30)


In [ ]:
dev_X.head()

,cocov1-can1-label,cocov1-can1-score,cocov1-can1-ymin,cocov1-can1-xmin,cocov1-can1-ymax,cocov1-can1-xmax,cocov1-can1-box-width,cocov1-can1-box-height,cocov1-can1-box-ratio,cocov1-can1-box-area,kitti-can1-label,kitti-can1-score,kitti-can1-ymin,kitti-can1-xmin,kitti-can1-ymax,kitti-can1-xmax,kitti-can1-box-width,kitti-can1-box-height,kitti-can1-box-ratio,kitti-can1-box-area,incep-can1-label,incep-can1-score,incep-can1-ymin,incep-can1-xmin,incep-can1-ymax,incep-can1-xmax,incep-can1-box-width,incep-can1-box-height,incep-can1-box-ratio,incep-can1-box-area
5726,2,0.999820,0.000000,0.000000,985.140326,569.375555,569.375555,985.140326,1.730212,560914.819315,2,0.999990,0.000000,0.000000,941.270164,536.563660,536.563660,941.270164,1.754256,505051.364572,2,0.999988,2.847549,4.005978,973.143090,536.115427,532.109449,970.295541,1.823489,516303.426171
6931,1,0.991420,444.221048,0.000000,560.755119,67.163571,67.163571,116.534071,1.735079,7826.844328,1,1.000000,447.904819,0.183711,560.053461,71.998185,71.814475,112.148641,1.561644,8053.895729,1,0.999988,447.097097,1.281633,561.344977,72.518619,71.236986,114.247880,1.603772,8138.674625
16,1,0.999823,409.773367,1123.098311,443.245373,1187.612412,64.514101,33.472006,0.518832,2159.416335,1,1.000000,406.725384,1136.619235,439.316492,1189.231938,52.612704,32.591107,0.619453,1714.706264,1,0.999996,407.646006,1138.056114,439.936572,1190.475788,52.419675,32.290567,0.616001,1692.660993
3516,1,0.999376,385.345674,912.463994,429.936870,985.110351,72.646357,44.591196,0.613812,3239.387920,1,0.999998,393.070566,906.202190,429.630215,985.205953,79.003762,36.559649,0.462758,2888.349805,1,0.999849,394.100230,905.887663,429.250260,984.619108,78.731445,35.150030,0.446455,2767.412640
5040,1,0.999587,400.023007,937.893502,461.179745,1008.920218,71.026716,61.156738,0.861039,4343.762248,1,0.999999,407.508088,952.065327,462.121592,1013.167311,61.101983,54.613503,0.893809,3336.993383,1,0.999995,407.601580,950.829748,461.936709,1016.016427,65.186679,54.335129,0.833531,3541.926568


In [ ]:
def run_lgb(train_X, train_y, val_X, val_y, test_X):

    lgtrain = lgb.Dataset(train_X, label=train_y, categorical_feature=['cocov1-can1-label','kitti-can1-label'])
    lgval = lgb.Dataset(val_X, label=val_y, reference=lgtrain, categorical_feature=['cocov1-can1-label','kitti-can1-label'])

    evals_result = {}
    params = {
        'boosting_type' : 'goss',
        'objective' : 'multiclass',
        'num_leaves' : 25,
        'num_class' : 3,
        'learning_rate' : 0.01,
        'bagging_seed' : 2018,
    }
    model = lgb.train(params, 
                      lgtrain, 
                      num_boost_round=661, 
                      valid_sets=[lgval], 
                      categorical_feature=['cocov1-can1-label','kitti-can1-label'],
                      early_stopping_rounds=100,
                      evals_result=evals_result)
    
    pred_test_y = model.predict(test_X, num_iteration=model.best_iteration)
    return pred_test_y, model, evals_result
    
    
pred_test, model, evals_result = run_lgb(dev_X, dev_y, val_X, val_y, test_X)
    

[1]	valid_0's multi_logloss: 0.814165
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's multi_logloss: 0.800432
[3]	valid_0's multi_logloss: 0.787188
[4]	valid_0's multi_logloss: 0.774404
[5]	valid_0's multi_logloss: 0.762025
[6]	valid_0's multi_logloss: 0.750033
[7]	valid_0's multi_logloss: 0.738397
[8]	valid_0's multi_logloss: 0.727094
[9]	valid_0's multi_logloss: 0.716105
[10]	valid_0's multi_logloss: 0.705414
[11]	valid_0's multi_logloss: 0.695005
[12]	valid_0's multi_logloss: 0.68485
[13]	valid_0's multi_logloss: 0.674949
[14]	valid_0's multi_logloss: 0.665282
[15]	valid_0's multi_logloss: 0.655844
[16]	valid_0's multi_logloss: 0.64662
[17]	valid_0's multi_logloss: 0.637607
[18]	valid_0's multi_logloss: 0.628797
[19]	valid_0's multi_logloss: 0.620166
[20]	valid_0's multi_logloss: 0.611723
[21]	valid_0's multi_logloss: 0.603456
[22]	valid_0's multi_logloss: 0.595355
[23]	valid_0's multi_logloss: 0.587415
[24]	valid_0's multi_logloss: 0.579638
[25]	valid_0

In [ ]:
from catboost import CatBoostClassifier
def run_cb(train_X, train_y, val_X, val_y, test_X): 
  clf = CatBoostClassifier(
      iterations=2392, 
      learning_rate=0.01, 
      #loss_function='CrossEntropy'
  )


  clf.fit(train_X, train_y, 
          cat_features=['cocov1-can1-label','kitti-can1-label'], 
          eval_set=(val_X, val_y), 
          verbose=True,
          early_stopping_rounds=100
  )
  pred_test_y=clf.predict(data=test_X)

  return pred_test_y, model, evals_result
    
    
pred_test, model, evals_result = run_cb(dev_X, dev_y, val_X, val_y, test_X)

0:	learn: 1.0801865	test: 1.0801286	best: 1.0801286 (0)	total: 48.1ms	remaining: 1m 54s
1:	learn: 1.0624006	test: 1.0622738	best: 1.0622738 (1)	total: 86.2ms	remaining: 1m 42s
2:	learn: 1.0448767	test: 1.0447127	best: 1.0447127 (2)	total: 145ms	remaining: 1m 55s
3:	learn: 1.0278689	test: 1.0276408	best: 1.0276408 (3)	total: 183ms	remaining: 1m 49s
4:	learn: 1.0113017	test: 1.0110481	best: 1.0110481 (4)	total: 232ms	remaining: 1m 50s
5:	learn: 0.9953192	test: 0.9950746	best: 0.9950746 (5)	total: 282ms	remaining: 1m 52s
6:	learn: 0.9793794	test: 0.9791299	best: 0.9791299 (6)	total: 332ms	remaining: 1m 53s
7:	learn: 0.9639530	test: 0.9636489	best: 0.9636489 (7)	total: 368ms	remaining: 1m 49s
8:	learn: 0.9490612	test: 0.9487085	best: 0.9487085 (8)	total: 416ms	remaining: 1m 50s
9:	learn: 0.9345036	test: 0.9340736	best: 0.9340736 (9)	total: 458ms	remaining: 1m 49s
10:	learn: 0.9203164	test: 0.9198769	best: 0.9198769 (10)	total: 517ms	remaining: 1m 51s
11:	learn: 0.9061807	test: 0.9057465	be

In [ ]:
test_X.head()

,cocov1-can1-label,cocov1-can1-score,cocov1-can1-ymin,cocov1-can1-xmin,cocov1-can1-ymax,cocov1-can1-xmax,cocov1-can1-box-width,cocov1-can1-box-height,cocov1-can1-box-ratio,cocov1-can1-box-area,kitti-can1-label,kitti-can1-score,kitti-can1-ymin,kitti-can1-xmin,kitti-can1-ymax,kitti-can1-xmax,kitti-can1-box-width,kitti-can1-box-height,kitti-can1-box-ratio,kitti-can1-box-area,incep-can1-label,incep-can1-score,incep-can1-ymin,incep-can1-xmin,incep-can1-ymax,incep-can1-xmax,incep-can1-box-width,incep-can1-box-height,incep-can1-box-ratio,incep-can1-box-area
0,0,1.0,105.870206,778.004116,599.913759,1198.261631,420.257514,494.043552,1.175573,207625.515156,0,1.000000,111.472683,782.244706,602.517044,1199.357172,417.112466,491.044361,1.177247,204820.724316,0,0.999922,114.727974,778.541677,597.486985,1199.429844,420.888167,482.759011,1.147001,203187.555194
1,0,1.0,107.833644,768.235507,598.237929,1182.570957,414.335451,490.404285,1.183592,203191.880580,0,1.000000,108.830568,767.190218,606.447963,1188.680802,421.490583,497.617395,1.180613,209741.046199,0,0.999919,118.921772,768.581806,588.000484,1186.015816,417.434010,469.078712,1.123719,195809.407618
2,0,1.0,99.043518,796.189221,601.950952,1224.586350,428.397129,502.907433,1.173928,215444.100425,0,1.000000,106.893600,797.865162,603.532913,1218.780195,420.915033,496.639313,1.179904,209042.953053,0,0.999927,108.871812,798.997039,594.052243,1221.967910,422.970871,485.180431,1.147078,205217.189787
3,0,1.0,98.499827,792.350204,604.166475,1221.011094,428.660889,505.666648,1.179643,216759.515104,0,0.999999,104.905449,794.580361,603.322039,1220.151818,425.571457,498.416590,1.171170,212111.874326,0,0.999941,112.773276,792.317748,592.349200,1215.951043,423.633296,479.575925,1.132054,203164.329518
4,0,1.0,99.756652,789.944644,605.358793,1219.450434,429.505790,505.602141,1.177172,217159.047226,0,1.000000,104.021847,790.766386,602.469264,1216.366877,425.600491,498.447417,1.171163,212139.465394,0,0.999928,111.382233,791.457046,593.168555,1211.594545,420.137499,481.786322,1.146735,202416.500180


In [ ]:
test_df['lgb_pred'] = np.argmax(pred_test,axis=1)
compare_df = test_df[['lgb_pred','cocov1-can1-label','kitti-can1-label']]

In [ ]:
compare_df[compare_df['lgb_pred'] != compare_df['cocov1-can1-label']].shape

(305, 3)

In [ ]:
print(compare_df[compare_df['lgb_pred'] != compare_df['cocov1-can1-label']])

      lgb_pred  cocov1-can1-label  kitti-can1-label
13           1                  0                 1
14           1                  0                 1
17           0                  2                 0
18           0                  1                 2
19           0                  2                 0
...        ...                ...               ...
2598         2                  1                 2
2599         2                  1                 2
2600         2                  1                 2
2601         2                  1                 2
2607         2                  1                 2

[305 rows x 3 columns]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
sub = df[['label']]
sub.head()

,label
0,0
1,0
2,0
3,0
4,0


In [ ]:
df = pd.DataFrame(pred_test, columns = ['label'])

In [ ]:
sub.columns = ['guid/image','label']
sub.to_csv('sub14.csv',index=False)

In [ ]:
sub.columns = ['label']
sub.to_csv('sub14.csv',index=False)

In [ ]:
goss= pd.read_csv('/content/lgbgoss25.csv',sep=',')
dart = pd.read_csv('/content/lgbdart.csv',sep=',')
gbdt =  pd.read_csv('/content/lgbgbdt.csv',sep=',')
cat = pd.read_csv('/content/sub14.csv',sep=',')
count=0
for i in range(2631):
  if goss['label'][i]==cat['label'][i]:
    count=count+1
  else:
    goss['label'][i]=0
sub = goss[['guid/image','label']]
sub.columns = ['guid/image','label']
sub.to_csv('sub14.csv',index=False)
print(count)

2631
